In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import json
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpStatus, lpSum, LpBinary, PULP_CBC_CMD

from sklearn.preprocessing import StandardScaler
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [5]:
# import json
# import os

# def load_graph(path):
#     with open(path, "r", encoding="utf-8") as f:
#         return json.load(f)

# def save_graph(path, graph_data):
#     os.makedirs(os.path.dirname(path), exist_ok=True)
#     with open(path, "w", encoding="utf-8") as f:
#         json.dump(graph_data, f, indent=2)

# def extract_prereq_edges(graph):
#     return [
#         (rel["source"].lower(), rel["target"].lower(), rel["weight"])
#         for rel in graph.get("relationships", [])
#         if rel["relationship"] == "prerequisite"
#     ]

# def build_skill_set(graph):
#     return set(skill["id"].lower() for skill in graph.get("skills", []))

# def update_prerequisites_from_folder(folder_path, output_folder="updated_skill_graph"):
#     json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]
#     graph_paths = [os.path.join(folder_path, f) for f in json_files]

#     graphs = {path: load_graph(path) for path in graph_paths}
#     skills_by_path = {path: build_skill_set(graphs[path]) for path in graph_paths}
#     prereqs_by_path = {path: extract_prereq_edges(graphs[path]) for path in graph_paths}

#     all_prereqs = set()
#     for prereqs in prereqs_by_path.values():
#         all_prereqs.update(prereqs)

#     for path in graph_paths:
#         current_skills = skills_by_path[path]
#         current_rels = graphs[path]["relationships"]
#         existing_edges = {
#             (rel["source"].lower(), rel["target"].lower())
#             for rel in current_rels
#             if rel["relationship"] == "prerequisite"
#         }

#         # Add missing prerequisite edges
#         for src, tgt, weight in all_prereqs:
#             if src in current_skills and tgt in current_skills and (src, tgt) not in existing_edges:
#                 graphs[path]["relationships"].append({
#                     "source": src,
#                     "target": tgt,
#                     "relationship": "prerequisite",
#                     "weight": weight
#                 })

#         # Save updated graph
#         filename = os.path.basename(path)
#         output_path = os.path.join(output_folder, filename)
#         save_graph(output_path, graphs[path])

# update_prerequisites_from_folder("../skill_graph", output_folder="updated_skill_graph")

In [6]:
# Helper Functions

from collections import defaultdict

def build_prereq_map(prereq_graph):
    prereq_map = defaultdict(set)
    for src, tgt in prereq_graph.edges():
        prereq_map[tgt].add(src)
    return prereq_map

def matchJobDomain(model, job_domains: dict, input_job_title: str, input_skills: list):
    """
    job_domains: dict of { job_domain_name: [skill1, skill2, ...] }
    input_job_title: str (e.g., "Data Analyst")
    input_skills: list of strings OR list of [skill, ...] tuples (only first element used)
    """

    # Normalize input skills if passed as list of lists (e.g., [["Python", 0.5, 0.6], ...])
    if isinstance(input_skills[0], (list, tuple)):
        input_skills = [skill[0] for skill in input_skills]

    # Encode input job title and skills
    input_title_emb = model.encode(input_job_title, convert_to_tensor=True)
    input_skill_embs = model.encode(input_skills, convert_to_tensor=True)

    best_match = None
    best_score = -1

    for domain_title, domain_skills in job_domains.items():
        # Encode domain title
        domain_title_emb = model.encode(domain_title, convert_to_tensor=True)
        title_score = float(util.cos_sim(input_title_emb, domain_title_emb))

        # Encode domain skills
        domain_skill_embs = model.encode(domain_skills, convert_to_tensor=True)
        sim_matrix = util.cos_sim(input_skill_embs, domain_skill_embs).cpu().numpy()

        # For each input skill, get the best match in the domain
        best_sim_per_skill = sim_matrix.max(axis=1)
        skill_score = best_sim_per_skill.mean() if len(best_sim_per_skill) > 0 else 0.0

        combined_score = 0.4 * title_score + 0.6 * skill_score

        if combined_score > best_score:
            best_score = combined_score
            best_match = {
                "matched_domain": domain_title,
                "title_score": round(title_score, 4),
                "skill_score": round(skill_score, 4),
                "combined_score": round(combined_score, 4),
            }

    return best_match

def extract_courses_for_skill(course_df, skill_name):
    """
    Extracts all courses that correspond to the given skill name from the course dataset.
    """
    return course_df[course_df["skill"].str.lower() == skill_name.lower()].copy()

def standardize_focus_scores(skill_list):
    """
    Standardizes the focus scores in the skill list so they sum to 1.
    Parameters:
    - skill_list (list): A list of skills with their focus-score and confidence level.
    Returns:
    - list: A standardized skill list where focus scores sum to 1.
    """
    # Extract focus scores
    focus_scores = np.array([entry[1] for entry in skill_list], dtype=np.float64)

    # Normalize so the sum equals 1
    total_focus = np.sum(focus_scores)
    if total_focus > 0:
        normalized_focus_scores = focus_scores / total_focus
    else:
        normalized_focus_scores = focus_scores  # If total is 0, keep them unchanged

    # Update skill list with standardized focus scores
    standardized_skill_list = [[skill[0], float(focus), skill[2]] for skill, focus in zip(skill_list, normalized_focus_scores)]

    return standardized_skill_list

import difflib

def compute_match_score(course_title, course_description, main_skill, all_skills, modules, prereq_graph):
    """
    Compute a course's match score based on:
    - Title match with main skill
    - Mentions of other skills from same module
    - Mentions of prerequisites of the main skill
    """

    # Normalize course text and skills
    title = course_title.lower()
    description = course_description.lower()
    full_text = f"{title} {description}"
    all_skill_names = [s[0].lower() if isinstance(s, (list, tuple)) else s.lower() for s in all_skills]
    main_skill = main_skill.lower()
    prereq_map = build_prereq_map(prereq_graph)

    score = 0.0

    # --- 1. Title Matching Boost ---
    if main_skill in title:
        score += 2.0  # Strong boost for exact match
    else:
        close_matches = difflib.get_close_matches(main_skill, [title], n=1, cutoff=0.8)
        if close_matches:
            score += 1.2  # Partial match

    # --- 2. Module Group Boost ---
    for module in modules:
        if main_skill in module["skills"]:
            same_module_skills = set(module["skills"]) - {main_skill}
            for skill in same_module_skills:
                if skill.lower() in full_text:
                    score += 0.5
                else:
                    matches = difflib.get_close_matches(skill.lower(), [full_text], n=1, cutoff=0.8)
                    if matches:
                        score += 0.3
            break

    # --- 3. Prerequisite Boost ---
    prereqs = prereq_map.get(main_skill, set())
    for prereq in prereqs:
        if prereq.lower() in full_text:
            score += 0.4
        else:
            matches = difflib.get_close_matches(prereq.lower(), [full_text], n=1, cutoff=0.8)
            if matches:
                score += 0.2

    return round(score, 3)


def compute_difficulty_scores(row, main_skill, skill_list):
    """
    Apply difficulty score computation to all courses in the dataset using the confidence level of the main skill.
    Parameters:
    - course_df (DataFrame): The dataset containing course information.
    - main_skill (str): The main skill for which courses are being evaluated.
    - skill_list (list): A list of skills with their focus-score and confidence level.
    Returns:
    - DataFrame: The original DataFrame with an added 'difficulty_score' column.
    """

    # Function to determine the ideal difficulty based on confidence
    def get_ideal_difficulty(confidence):
        if confidence >= 0.5:
            diff = 3
        elif confidence >= 0.3:
            diff = 2
        elif confidence >= 0.2:
            diff = 1.5
        elif confidence >= 0.1:
            diff = 1
        else:
            diff = 0
        return diff

    # Function to compute difficulty score for a single course row
    def get_difficulty_score(course_difficulty, user_confidence, is_pro_certificate):
        ideal_difficulty = get_ideal_difficulty(user_confidence)
        difficulty_penalty = 1 * np.abs(ideal_difficulty - course_difficulty)
        if is_pro_certificate and user_confidence >= 0.6:
            certificate_score = 10
        elif is_pro_certificate and user_confidence <= 0.3:
            certificate_score = -10
        else:
            certificate_score = 0
        return 1-difficulty_penalty + certificate_score

    
    main_skill_confidence = next((conf for skill, focus, conf in skill_list if skill.lower() == main_skill.lower()), 0)
    return get_difficulty_score(
        course_difficulty=row["difficulty_numeric"],
        user_confidence=main_skill_confidence,
        is_pro_certificate=(row["course_type"] == "Certificate")
    )

# Function to solve the ILP for selecting courses with a dynamic duration constraint
def solve_course_selection_pulp(course_df, skill, D_ideal, alpha, beta, lambda_, gamma):
    """
    Solves the ILP for selecting courses that maximize skill match and difficulty score while minimizing price,
    while ensuring the total selected duration stays within ±10% of the ideal duration.

    Parameters:
    - course_df (DataFrame): The dataset containing courses with relevant fields.
    - skill (str): The skill being optimized in this iteration.
    - skill_list (list): A list of skills with their focus-score and confidence level.
    - D_ideal (float): The ideal duration for the selected courses.
    - alpha, gamma, lambda_, beta: Weight parameters for optimization.

    Returns:
    - Selected courses as a list of course indices.
    """

    # Extract relevant courses for the given skill
    relevant_courses = course_df[course_df["skill"].str.lower() == skill.lower()].copy()
    if relevant_courses.empty:
        return []  # No courses available for this skill

    # Define decision variables (binary: select or not)
    x = {i: LpVariable(f"x_{i}", cat="Binary") for i in relevant_courses.index}

    # Define the ILP problem
    problem = LpProblem("Course_Selection", LpMaximize)

    match_score = lpSum(relevant_courses.loc[i, "match_score"] * x[i] for i in relevant_courses.index)
    difficulty_score = lpSum(relevant_courses.loc[i, "difficulty_score"] * x[i] for i in relevant_courses.index)
    wilson_score = lpSum(relevant_courses.loc[i, "wilson_score"] * x[i] for i in relevant_courses.index)
    price_score = lpSum(relevant_courses.loc[i, "price"] * x[i] for i in relevant_courses.index)
    number_score = lpSum(x[i] for i in relevant_courses.index)

    # Objective Function: Maximize average skill match, difficulty, and review score while minimizing price
    problem += (
        match_score +
        alpha * difficulty_score +
        beta * wilson_score -
        lambda_ * price_score -
        gamma *  number_score
    )
    

    # Skill Coverage Constraint: At least one course must be selected
    problem += lpSum(x[i] for i in relevant_courses.index) >= 1

    # Duration Constraint: Total selected duration must be within ±10% of the ideal duration
    duration_tolerance = 0.1 * D_ideal
    problem += lpSum(relevant_courses.loc[i, "duration"] * x[i] for i in relevant_courses.index) <= (D_ideal + duration_tolerance)
    problem += lpSum(relevant_courses.loc[i, "duration"] * x[i] for i in relevant_courses.index) >= (D_ideal - duration_tolerance)

    # Solve the ILP
    problem.solve()

    # Extract selected courses
    selected_courses = [i for i in relevant_courses.index if x[i].varValue == 1]

    return selected_courses

def suggest_courses(
    embedding_model,
    course_df,
    skill_list,
    total_weeks,
    weekly_hours,
    job_title,
    module_skills,
    prereq_graph,
    portion=0.8,
    alpha=0.5,
    beta=0.5,
    lambda_=0.5,
    gamma=1
):
    result = {}
    skill_list = standardize_focus_scores(skill_list)
    scaler = StandardScaler()

    # Precompute embeddings
    skill_embeddings = {
        skill[0].lower(): embedding_model.encode(skill[0].lower(), convert_to_tensor=True)
        for skill in skill_list
    }
    job_title_embedding = embedding_model.encode(job_title.lower(), convert_to_tensor=True)

    for [skill, focus, confidence] in skill_list:
        main_skill = skill
        D_ideal = portion * total_weeks * weekly_hours * focus

        # Extract + preprocess courses
        courses = extract_courses_for_skill(course_df, main_skill)
        if courses.empty:
            result[main_skill] = []
            continue

        courses["description_embedding"] = courses["description"].apply(
            lambda text: embedding_model.encode(text, convert_to_tensor=True)
        )
        courses["title_embedding"] = courses["title"].apply(
            lambda text: embedding_model.encode(text, convert_to_tensor=True)
        )
        courses["match_score"] = courses.apply(
            lambda row: compute_match_score(
                row["title"],
                row["description"],
                main_skill,
                skill_list,
                module_skills,
                prereq_graph
            ),
            axis=1
        )
        courses["difficulty_score"] = courses.apply(
            lambda row: compute_difficulty_scores(row, main_skill, skill_list), axis=1
        )

        # Normalize scores
        for col in ["match_score", "difficulty_score", "price", "wilson_score"]:
            courses[col] = scaler.fit_transform(courses[[col]])

        # Solve ILP
        selected_indices = solve_course_selection_pulp(
            courses, main_skill, D_ideal, alpha, beta, lambda_, gamma
        )

        if selected_indices:
            selected_courses = course_df.loc[selected_indices].to_dict(orient="records")
        else:
            # ⛑️ Pick fallback: shortest duration course
            fallback = course_df.sort_values("duration").head(1)
            selected_courses = fallback.to_dict(orient="records")

        result[main_skill] = selected_courses

    return result

In [7]:
import json
import networkx as nx
from collections import defaultdict
import math


def load_knowledge_graph(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def parse_prerequisite_edges(graph_path, input_skill_set):
    knowledge_graph = load_knowledge_graph(graph_path)
    prereq_edges = []
    for rel in knowledge_graph.get("relationships", []):
        if rel["relationship"] == "prerequisite":
            src, tgt = rel["source"].lower(), rel["target"].lower()
            if src in input_skill_set and tgt in input_skill_set:
                prereq_edges.append((src, tgt, rel["weight"]))
    return prereq_edges


def parse_association_weights(graph_path, input_skill_set):
    knowledge_graph = load_knowledge_graph(graph_path)
    association_scores = {}
    for rel in knowledge_graph.get("relationships", []):
        if rel["relationship"] == "association":
            src, tgt = rel["source"].lower(), rel["target"].lower()
            if src in input_skill_set and tgt in input_skill_set:
                weight = rel.get("weight", 0)
                association_scores[(src, tgt)] = weight
                association_scores[(tgt, src)] = weight
    return association_scores


def build_prereq_graph_from_edges(prereq_edges):
    G = nx.DiGraph()
    for src, tgt, weight in prereq_edges:
        G.add_edge(src, tgt, weight=weight)
    return G


def break_cycles(graph):
    while True:
        try:
            cycle = nx.find_cycle(graph, orientation="original")
            weakest = min(cycle, key=lambda e: graph.get_edge_data(e[0], e[1]).get("weight", 1.0))
            graph.remove_edge(weakest[0], weakest[1])
        except nx.exception.NetworkXNoCycle:
            break


def topological_sort_with_priorities(prereq_graph, input_skill_set):
    break_cycles(prereq_graph)

    try:
        topo_order = list(nx.topological_sort(prereq_graph))

        skill_scores = {}
        for skill in topo_order:
            incoming = list(prereq_graph.in_edges(skill, data=True))
            prereq_weight = sum(data['weight'] for _, _, data in incoming) / len(incoming) if incoming else 0
            skill_scores[skill] = prereq_weight

        refined_order = sorted(topo_order, key=lambda s: skill_scores[s])

        for skill in input_skill_set:
            if skill not in refined_order:
                refined_order.append(skill)

        return refined_order

    except nx.NetworkXUnfeasible:
        return list(prereq_graph.nodes())


def group_skills_by_association(topo_order, association_scores, input_skills, prereq_graph, threshold=0.3):
    groups = []
    skill_to_module = {}
    prereq_map = defaultdict(set)

    if not isinstance(prereq_graph, nx.Graph):
        prereq_graph = build_prereq_graph_from_edges(prereq_graph)

    for src, tgt in prereq_graph.edges():
        prereq_map[tgt].add(src)

    for skill in topo_order:
        max_prereq_module = max(
            [skill_to_module[pre] for pre in prereq_map[skill] if pre in skill_to_module],
            default=-1
        )

        best_group = None
        best_score = 0

        for idx in range(max_prereq_module + 1, len(groups)):
            group = groups[idx]
            if len(group) >= 3:
                continue
            score = sum(association_scores.get((skill, other), 0) for other in group)
            avg_score = score / len(group) if group else 0
            if avg_score >= threshold:
                best_group = idx
                best_score = avg_score
                break

        if best_group is not None:
            groups[best_group].append(skill)
            skill_to_module[skill] = best_group
        else:
            groups.append([skill])
            skill_to_module[skill] = len(groups) - 1

    return groups


def assign_module_durations(skill_groups, input_skills, total_hours):
    skill_focus_map = {s[0].lower(): s[1] for s in input_skills}
    module_durations = []

    for group in skill_groups:
        durations = [round(skill_focus_map.get(skill, 0) * total_hours, 1) for skill in group]
        module_durations.append(durations)

    modules = []
    for i, (skills, durations) in enumerate(zip(skill_groups, module_durations), 1):
        modules.append({
            "module": i,
            "skills": skills,
            "duration": durations
        })
    return modules

In [8]:
# === Step 1: Load Job Domains ===
with open("../job_domain_skills.json", "r", encoding="utf-8") as f:
    job_domains = json.load(f)

# === Step 2: User Inputs ===
input_skills = [
    ["python", 0.3, 0.6], 
    ["machine learning", 0.1, 0.3],
    ["pandas", 0.05, 0.1],
    ["sql", 0.1, 0.3],
    ["git", 0.01, 0.1],
    ["data analysis", 0.15, 0.4],
    ["nlp", 0.09, 0],
    ["deep learning", 0.05, 0.1],
    ["excel", 0.1, 0.4], 
    ["pytorch", 0.05, 0]
]
input_job_title = "Data Analyst"
total_weeks = 10
weekly_hours = 20
total_hours = total_weeks * weekly_hours

# === Step 3: Match Domain ===
best_match = matchJobDomain(embedding_model, job_domains, input_job_title, input_skills)
matched_domain = best_match["matched_domain"]
print("Matched job domain:", matched_domain)

# === Step 4: Graph Paths & Inputs ===
graph_path = f"./updated_skill_graph/{matched_domain}.json"
input_skill_set = set(s[0].lower() for s in input_skills)

# === Step 5: Extract Graph Info ===
prereq_edges = parse_prerequisite_edges(graph_path, input_skill_set)
assoc_scores = parse_association_weights(graph_path, input_skill_set)

# === Step 6: Build Prereq Graph & Topo Sort ===
prereq_graph = build_prereq_graph_from_edges(prereq_edges)
ordered_skills = topological_sort_with_priorities(prereq_graph, input_skill_set)

# === Step 7: Group & Allocate Time ===
skill_groups = group_skills_by_association(
    ordered_skills, assoc_scores, input_skills, prereq_graph, threshold=0.2
)
final_modules = assign_module_durations(skill_groups, input_skills, total_hours)

final_modules

Matched job domain: Data Scientist


[{'module': 1, 'skills': ['python'], 'duration': [60.0]},
 {'module': 2, 'skills': ['machine learning'], 'duration': [20.0]},
 {'module': 3, 'skills': ['nlp'], 'duration': [18.0]},
 {'module': 4,
  'skills': ['pytorch', 'deep learning'],
  'duration': [10.0, 10.0]},
 {'module': 5, 'skills': ['data analysis'], 'duration': [30.0]},
 {'module': 6, 'skills': ['pandas', 'git'], 'duration': [10.0, 2.0]},
 {'module': 7, 'skills': ['sql'], 'duration': [20.0]},
 {'module': 8, 'skills': ['excel'], 'duration': [20.0]}]

In [9]:
# Load the Coursera and Udemy course df
coursera_file_path = f"../cleaned_df/coursera_cleaned_df/{matched_domain}.csv"
udemy_file_path = f"../cleaned_df/udemy_cleaned_df/{matched_domain}.csv"
new_column_order = [
    "skill", "link", "image_link", "partner", "title", "description", "rating",
    "num_review", "duration", "price", "difficulty_numeric", "course_type", "wilson_score", 
]
# Read CSV files into Pandas DataFrames
coursera_df = pd.read_csv(coursera_file_path)[new_column_order]
udemy_df = pd.read_csv(udemy_file_path)[new_column_order]
course_df = pd.concat([coursera_df, udemy_df], axis=0).reset_index()
course_df["difficulty"] = course_df["difficulty_numeric"]
course_df["price"] = round(course_df["price"], 0) + 0.99

In [10]:
suggestions = suggest_courses(
    embedding_model=embedding_model, 
    course_df=course_df, 
    skill_list=input_skills, 
    total_weeks=total_weeks, 
    weekly_hours=weekly_hours, 
    job_title=matched_domain, 
    module_skills=final_modules,
    prereq_graph=prereq_graph,
    portion=1
)

In [ ]:
from datetime import datetime, timedelta, time
import pandas as pd
import math

# === CONFIGURATION ===
USER_ID = 1
START_DATE = datetime.strptime("2025-04-10", "%Y-%m-%d")
WEEKLY_HOURS = 20
LEARNING_DAYS = {
    "Monday": True,
    "Tuesday": True,
    "Wednesday": True,
    "Thursday": False,
    "Friday": False,
    "Saturday": True,
    "Sunday": True,
}

LUNCH_BREAK = (time(11, 0), time(12, 0))
DINNER_BREAK = (time(17, 0), time(18, 0))
BREAK_OPTIONS = [timedelta(minutes=15), timedelta(minutes=30)]
MIN_CHUNK = 1.0  # in hours

# === DAILY HOURS DISTRIBUTOR ===
def distribute_weekly_hours(weekly_hours, learning_days):
    total_days = sum(learning_days.values())
    weekend_days = int(learning_days.get("Saturday", False)) + int(learning_days.get("Sunday", False))
    weekday_days = total_days - weekend_days

    if weekend_days == 0:
        base_hour = weekly_hours / total_days
        return {day: round(base_hour * 2) / 2 for day, v in learning_days.items() if v}

    weekday_hours = weekly_hours - weekend_days * 1
    if weekday_days * 8 < weekday_hours:
        weekday_hours = weekday_days * 8
        weekend_hours = weekly_hours - weekday_hours
        base_weekend = weekend_hours / weekend_days
    else:
        base_weekend = 1

    base_weekday = weekday_hours / weekday_days
    all_days = {}
    for day, v in learning_days.items():
        if not v:
            continue
        if day in ["Saturday", "Sunday"]:
            all_days[day] = round(base_weekend * 2) / 2
        else:
            all_days[day] = round(base_weekday * 2) / 2

    return all_days

# === HELPER: Round-robin hour splitting ===
def split_hours_round_robin(total_hours, num_blocks):
    blocks = [0.0] * num_blocks
    i = 0
    while total_hours >= 0.5:
        blocks[i] += 0.5
        total_hours -= 0.5
        i = (i + 1) % num_blocks
    return blocks

from datetime import datetime, timedelta, time
import pandas as pd
import math

# === CONFIG ===
start_hour = 8
end_hour = 21
slot_duration = 15  # minutes
slots_per_hour = 60 // slot_duration
total_slots = (end_hour - start_hour) * slots_per_hour
daily_hours = 6
target_minutes = daily_hours * 60

# === Time slots ===
time_slots = []
current_minutes = start_hour * 60
for i in range(total_slots):
    start = current_minutes
    end = start + slot_duration
    midpoint = (start + end) // 2
    time_slots.append({
        "index": i,
        "start_min": start,
        "end_min": end,
        "mid_min": midpoint,
        "comfort": -abs(midpoint - 12 * 60)  # closer to noon = better
    })
    current_minutes += slot_duration

# === Forbidden time ranges (11–12 and 17–18)
forbidden_ranges = [(11*60, 12*60), (17*60, 18*60)]
forbidden_slots = [s["index"] for s in time_slots if any(start <= s["mid_min"] < end for start, end in forbidden_ranges)]

# === ILP Model ===
model = LpProblem("DailyBlockComfort", LpMaximize)
x = {s["index"]: LpVariable(f"x_{s['index']}", cat=LpBinary) for s in time_slots}

# Objective: maximize comfort
model += lpSum(x[s["index"]] * s["comfort"] for s in time_slots)

# Learning time constraint
model += lpSum(x[s["index"]] * slot_duration for s in time_slots) == target_minutes

# No learning during forbidden slots
for idx in forbidden_slots:
    model += x[idx] == 0

# At least 4, at most 12 consecutive slots with at least 1 break slot between
# (soft version here — can be made stricter with auxiliary variables)

# Solve
model.solve(PULP_CBC_CMD(msg=0))

# Output selected blocks
selected_blocks = []
for s in time_slots:
    if x[s["index"]].varValue == 1:
        st = time(s["start_min"] // 60, s["start_min"] % 60)
        et = time(s["end_min"] // 60, s["end_min"] % 60)
        selected_blocks.append((st, et))

for block in selected_blocks:
    print(block)


Exception: Cannot fit required hours into day schedule

In [ ]:
daily_hours_map

{'Monday': 6.0,
 'Tuesday': 6.0,
 'Wednesday': 6.0,
 'Saturday': 1.0,
 'Sunday': 1.0}